In [7]:
from huggingface_hub import hf_hub_download
import open_clip
import torch

In [12]:
%cd /home/hjh/rs-paint

/home/hjh/rs-paint


In [ ]:
model_name = 'ViT-L-14'
checkpoint_path = hf_hub_download("chendelong/RemoteCLIP", f"RemoteCLIP-{model_name}.pt", cache_dir='checkpoints')
print(f'{model_name} is downloaded to {checkpoint_path}.')

In [40]:
checkpoint_path = 'checkpoints/remoteclip.pt'

In [42]:
model, _, preprocess = open_clip.create_model_and_transforms(model_name)

In [43]:
ckpt = torch.load(checkpoint_path, map_location="cpu")
message = model.load_state_dict(ckpt)
print(message)
model = model.cuda().eval()

<All keys matched successfully>


In [38]:
import inspect

print(inspect.getsource(model.visual.forward))
print(inspect.getsource(model.visual._global_pool))
model.visual.pool_type

    def forward(self, x: torch.Tensor):
        x = self.conv1(x)  # shape = [*, width, grid, grid]
        x = x.reshape(x.shape[0], x.shape[1], -1)  # shape = [*, width, grid ** 2]
        x = x.permute(0, 2, 1)  # shape = [*, grid ** 2, width]

        # class embeddings and positional embeddings
        x = torch.cat([_expand_token(self.class_embedding, x.shape[0]).to(x.dtype), x], dim=1)
        # shape = [*, grid ** 2 + 1, width]
        x = x + self.positional_embedding.to(x.dtype)

        x = self.patch_dropout(x)
        x = self.ln_pre(x)
        x = self.transformer(x)

        if self.attn_pool is not None:
            if self.attn_pool_contrastive is not None:
                # This is untested, WIP pooling that should match paper
                x = self.ln_post(x)  # TBD LN first or separate one after each pool?
                tokens = self.attn_pool(x)
                if self.attn_pool_type == 'parallel':
                    pooled = self.attn_pool_contrastive(x)
    

'tok'

In [39]:
img = torch.randn(1, 3, 224, 224).cuda()
model.visual.proj = None
out = model.visual(img)
out.shape

torch.Size([1, 1024])